In [8]:
import sys
sys.path.append("../..")
import torch
from pathlib import Path
import scipy
import numpy as np
from lru.architectures import DLRU, DLRUConfig
from lru.reduction import lru_reduction_pipeline
import matplotlib.pyplot as plt
import torchid.metrics  # pip install pytorch-ident
import copy

In [9]:
# Load data
data_folder = ("F16GVT_Files", "BenchmarkData")

# file_name = "F16Data_SineSw_Level5.mat"
#
# file_name = "F16Data_FullMSine_Level7.mat"
# file_name = "F16Data_FullMSine_Level5.mat"
# file_name = "F16Data_FullMSine_Level4_Validation.mat"
file_name = "F16Data_FullMSine_Level6_Validation.mat"
# file_name = "F16Data_SineSw_Level6_Validation.mat"
file_path = Path(*data_folder) / file_name
data = scipy.io.loadmat(file_path)

In [10]:
run = "ckpt_large_reg_hankel"
#run = "ckpt_large_reg_hankel_cc"
#run = "ckpt_large_reg_modal"  # lasso on eigs abs val
#run = "ckpt_large_no_reg"
# run = "ckpt_large_no_reg_last"
# run = "ckpt_small_no_reg"
# run = "ckpt_small_reg"

In [11]:
#reduction_method = "balanced_truncation"
#reduction_method = "balanced_singular_perturbation"
reduction_method = "modal_truncation"
#reduction_method = "modal_singular_perturbation" 

In [12]:
ckpt = torch.load(Path("ckpt")/ f"{run}.pt", map_location="cpu")
cfg = ckpt["cfg"]
scaler_u = ckpt["scaler_u"]
scaler_y = ckpt["scaler_y"]

/Users/marco.forgione/anaconda3/envs/dev/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.4.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
u_test = data["Force"].T  # Or force
y_test = data["Acceleration"].T

In [14]:
# Load model
config = DLRUConfig(
    d_model=cfg.d_model, d_state=cfg.d_state, n_layers=cfg.n_layers, ff=cfg.ff
)
model = DLRUConfig(1, 3, config)
model.load_state_dict(ckpt["model"])

AttributeError: 'DLRUConfig' object has no attribute 'load_state_dict'

In [ ]:
MODES = []
FIT_MEAN = []
for modes in range(cfg.d_state, 0, -1):

    model_reduced = copy.deepcopy(model)

    for block in model_reduced.blocks:

        # reduction pipeline
        ss_params = block.lru.ss_params()
        ss_params = [param.detach().numpy() for param in ss_params]
        lambdas, B, C, D = ss_params
        lambdas_red, B_red, C_red, D_red = lru_reduction_pipeline(lambdas, B, C, D,  modes=modes, method=reduction_method)


        params_red = [lambdas_red.astype(np.complex64), B_red.astype(np.complex64), C_red.astype(np.complex64), D_red.astype(np.float32)]
        params_red = [torch.tensor(param_red) for param_red in params_red]
        block.lru.replace_ss_params(*params_red)

    ut = torch.tensor(scaler_u.transform(u_test)).unsqueeze(0).float()
    with torch.no_grad():
        #    y_test_hat = model(ut, mode="scan").squeeze(0).to("cpu").numpy()
        y_test_hat = model_reduced(ut, mode="scan").squeeze(0).to("cpu").numpy()

    y_test_hat = scaler_y.inverse_transform(y_test_hat)

    fit = torchid.metrics.fit_index(y_test, y_test_hat).mean()
    MODES.append(modes)
    FIT_MEAN.append(fit)

MODES = np.array(MODES)
FIT_MEAN = np.array(FIT_MEAN)


In [ ]:
#FIT_THRESHOLD = 83.25865396027804*0.99 # 1% less than the worst
FIT_THRESHOLD = FIT_MEAN[0] * 0.99
MIN_ORDER = MODES[FIT_MEAN > FIT_THRESHOLD].min()
fig, ax = plt.subplots()
ax.plot(MODES, FIT_MEAN)
ax.axhline(FIT_THRESHOLD, color="red")
ax.axvline(MIN_ORDER, color="black")
ax.invert_xaxis()
ax.grid(True)